# Homework 3
Student Name: Huynh Quang Nguyen Vo

CWID: A20446163

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

##
# Parameters
#
months = np.linspace(1, 6, 6)
demands = [1055, 700, 1645, 1600, 1095, 845]
production_capacities = [1000, 300]
production_costs = [10, 15]
inventory_cost = 2
backordered_cost = 5
inventories = 0 
backorders = 0 

indices = range(1,len(demands)+1)

try:

    m = gp.Model('ProductionPlanning')
    
    ##
    # Decision variables: 
    #    
    x = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='units_produced-rt')
    y = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='units_produced_ot')
    z = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='units_carried')
    w = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='back_order_units')
    
    
    ##
    # Objective function:
    #
    m.setObjective(gp.quicksum([production_costs[0] * x[i] + production_costs[1] * y[i] + inventory_cost * z[i]
                             + backordered_cost * w[i] for i in indices]), GRB.MINIMIZE)
    
    ##
    # Constraints:
    #
    m.addConstr(z[1] == inventories)
    m.addConstr(w[1] == backorders)
    m.addConstr(z[6] == inventories)
    m.addConstr(w[6] == backorders)
    
    for i in indices:
        if i == 1:
            m.addConstr(x[i] + y[i]- z[i] == demands[i-1] - w[i])
        else:
            m.addConstr(x[i] + y[i] + z[i-1] - z[i] == demands[i-1] + w[i-1] - w[i])
    
    for i in indices:
        m.addConstr(x[i] <= production_capacities[0])
    
        m.addConstr(y[i] <= production_capacities[1])
    
##
# Solve the linear program:
#     
    m.write('ProductionPlanning.lp')
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        for v in m.getVars():
            print('%s %g' % (v.varName, v.x))
            print('Obj: %g' % m.objVal)
    elif m.status == GRB.INFEASIBLE:
        print('LP is infeasible.')
    elif m.status == GRB.UNBOUNDED:
        print('LP is unbounded.')
except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))
except AttributeError:
        print('Encountered an attribute error') 

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 22 rows, 24 columns and 50 nonzeros
Model fingerprint: 0x8b60375f
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 2e+03]
Found heuristic solution: objective 93270.000000
Presolve removed 17 rows and 6 columns
Presolve time: 0.00s
Presolved: 5 rows, 18 columns, 26 nonzeros
Found heuristic solution: objective 92026.000000
Variable types: 0 continuous, 18 integer (0 binary)
Found heuristic solution: objective 92007.000000

Root relaxation: objective 7.681000e+04, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Cur